# API for Game Data (Not Needed)

In [26]:
import requests
import pandas as pd

def get_nfl_games_regular_and_postseason(year: int) -> pd.DataFrame:
    """
    Return a DataFrame of all NFL Regular Season + Postseason games for a given year.
    Columns:
      - game_id
      - date
      - home_team
      - away_team
    """
    HEADER = {"User-Agent": "xcs5hg@virginia.edu"}

    def get_json(url: str):
        r = requests.get(url, headers=HEADER)
        r.raise_for_status()
        return r.json()

    # 1) Get season metadata
    season_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{year}"
    season_data = get_json(season_url)

    # 2) Find Regular Season + Postseason objects (your idea)
    season_parts = {}  # {"Regular Season": {...}, "Postseason": {...}}

    for t in season_data["types"]["items"]:
        t_data = get_json(t["$ref"])
        name = t_data.get("name")
        if name in ["Regular Season", "Postseason"]:
            season_parts[name] = t_data

    if not season_parts:
        raise ValueError(f"No Regular/Postseason types found for {year}")

    # 3) Collect all game (event) URLs from both Regular + Postseason
    event_urls = []

    for part_name, part in season_parts.items():
        weeks_url = part["weeks"]["$ref"]
        weeks_data = get_json(weeks_url)
        week_items = weeks_data.get("items", [])

        for w in week_items:
            week = get_json(w["$ref"])
            events_url = week["events"]["$ref"]
            events_data = get_json(events_url)
            event_urls.extend(e["$ref"] for e in events_data.get("items", []))

    # 4) Loop over events and extract game_id, date, home_team, away_team
    records = []

    for event_url in event_urls:
        event = get_json(event_url)

        game_id = event.get("id")
        date = event.get("date")

        comp = event["competitions"][0]
        competitors = comp.get("competitors", [])

        home_team = None
        away_team = None

        for c in competitors:
            side = c.get("homeAway")  # 'home' or 'away'
            team_obj = c.get("team", {})

            # Team might be a $ref or an embedded object
            if "$ref" in team_obj:
                team_data = get_json(team_obj["$ref"])
            else:
                team_data = team_obj

            team_name = (
                team_data.get("displayName")
                or team_data.get("name")
                or team_data.get("abbreviation")
            )

            if side == "home":
                home_team = team_name
            elif side == "away":
                away_team = team_name

        records.append(
            {
                "game_id": game_id,
                "date": date,
                "home_team": home_team,
                "away_team": away_team,
            }
        )

    return pd.DataFrame(records)

In [30]:
df_games_2024

NameError: name 'df_games_2024' is not defined

In [31]:
df_games_2024 = build_nfl_games_df(2024)
print(df_games_2024.head())
print(len(df_games_2024))

NameError: name 'build_nfl_games_df' is not defined

# Odds API (Not exactly what we want)

In [56]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_game_odds_and_probabilities(game_id: int):

    base = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl"
    
    def get_json(url):
        r = requests.get(url, headers=header)
        r.raise_for_status()
        return r.json()

    event_url = f"{base}/events/{game_id}"
    event = get_json(event_url)

    comp = event["competitions"][0]
    comp_id = comp["id"]


    probs_url = (
        f"{base}/events/{game_id}/competitions/{comp_id}/probabilities?limit=500"
    )
    probabilities = get_json(probs_url)

    odds_url = (
        f"{base}/events/{game_id}/competitions/{comp_id}/odds"
    )
    odds = get_json(odds_url)

    return {
        "game_id": game_id,
        "competition_id": comp_id,
        "event": event,
        "probabilities": probabilities,
        "odds": odds,
    }

In [59]:
odds = get_game_odds_and_probabilities(401671789)

In [71]:
pd.json_normalize(odds['odds']['items'])

,$ref,details,overUnder,spread,overOdds,underOdds,links,moneylineWinner,spreadWinner,provider.$ref,...,bettingOdds.teamOdds.preMatchSpreadAway.betSlipUrl,bettingOdds.teamOdds.preMatchTotalUnder.oddId,bettingOdds.teamOdds.preMatchTotalUnder.value,bettingOdds.teamOdds.preMatchTotalUnder.betSlipUrl,bettingOdds.teamOdds.preMatchTotalHandicap.oddId,bettingOdds.teamOdds.preMatchTotalHandicap.value,bettingOdds.teamOdds.preMatchTotalHandicap.betSlipUrl,bettingOdds.teamOdds.preMatchSpreadHandicapHome.oddId,bettingOdds.teamOdds.preMatchSpreadHandicapHome.value,bettingOdds.teamOdds.preMatchSpreadHandicapHome.betSlipUrl
0,http://sports.core.api.espn.com/v2/sports/foot...,KC -2.5,45.5,-2.5,-115.0,-105.0,"[{'language': 'en-US', 'rel': ['home', 'deskto...",False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://sports.core.api.espn.com/v2/sports/foot...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,https://www.bet365.com/dl/sportsbookredirect?a...,484455341,10/11,https://www.bet365.com/dl/sportsbookredirect?a...,484455341,46.5,https://www.bet365.com/dl/sportsbookredirect?a...,484455319,-2.5,https://www.bet365.com/dl/sportsbookredirect?a...
2,http://sports.core.api.espn.com/v2/sports/foot...,KC -10.5,48.5,-10.5,185.0,-275.0,"[{'language': 'en-US', 'rel': ['home', 'deskto...",False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_scoreboard(start_date: str, end_date: str | None = None, limit: int = 1000):

    base_url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
    header = {"User-Agent": "xcs5hg@virginia.edu"}


    # Build dates parameter: either 'YYYYMMDD' or 'YYYYMMDD-YYYYMMDD'
    if end_date is not None:
        dates_param = f"{start_date}-{end_date}"
    else:
        dates_param = start_date  # can be 'YYYY', 'YYYYMMDD', etc.

    params = {
        "limit": limit,
        "dates": dates_param,
        # optional cache-buster if you want:
        # "_": "1577413600",
    }

    r = requests.get(base_url, headers=header, params=params)
    r.raise_for_status()
    return r.json()

# Athletes API

In [21]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_active_athletes(max_athletes: int = 10000):
    """
    Return a list of $ref URLs for all active NFL athletes (up to max_athletes),
    following ESPN's pagination.
    """
    url = "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes"
    params = {
        "active": "true",
        "limit": 20000,  
    }
    header = {"User-Agent": "xcs5hg@virginia.edu"}

    records = []

    while url and len(records) < max_athletes:
        resp = requests.get(url, headers=header, params=params)
        resp.raise_for_status()
        data = resp.json()

        for item in data.get("items", []):
            records.append(item["$ref"])
            if len(records) >= max_athletes:
                break

        next_ref = data.get("next", {}).get("$ref")
        if next_ref and len(records) < max_athletes:
            url = next_ref
            params = None 
        else:
            url = None
    
    players = pd.DataFrame()
    
    for i in range(0,50):
        player = requests.get(records[i], headers=header).json()
        player = pd.json_normalize(player)  
        #player = player[['id','fullName','displayName']]
        
        players = pd.concat([players, player], ignore_index=True)      
    
    return players

In [22]:
data = get_active_athletes()

IndexError: list index out of range

In [ ]:
len(data)

In [238]:
data

,id,fullName,displayName
0,4429202,Israel Abanikanda,Israel Abanikanda
1,2576336,Ameer Abdullah,Ameer Abdullah
2,3049325,Khalid Abdullah,Khalid Abdullah
3,4360807,Yasir Abdullah,Yasir Abdullah
4,3915373,Micah Abernathy,Micah Abernathy
5,15972,Oday Aboushi,Oday Aboushi
6,4427563,Micah Abraham,Micah Abraham
7,3728305,Johnathan Abram,Johnathan Abram
8,4569234,Keshunn Abram,Keshunn Abram
9,4360643,Delrick Abrams,Delrick Abrams


# Play by play API 

In [52]:
import requests

def get_game_plays(game_id: int):

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    event_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}"
    )
    event = requests.get(event_url, headers=header).json()

    competition = event["competitions"][0]
    comp_id = competition["id"]

    plays_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}/competitions/{comp_id}/plays?limit=500"
    )

    plays = requests.get(plays_url, headers=header).json()
    
    record = []
    
    for item in plays["items"]:
        record.append(item)
        
    keep_cols = [
    "text",
    "period.number",
    "clock.value",
    "start.down",
    "start.distance",
    "start.yardLine",
    "start.yardsToEndzone",
    "end.yardsToEndzone",
    "homeScore",
    "awayScore",
    "statYardage",
    "scoringPlay",
    "scoreValue",
    "type.text",
    "type.abbreviation"]
    
    data = pd.json_normalize(record)
    data = data[keep_cols]  
    
    
    return data

In [53]:
plays = get_game_plays(401671789)
plays


,text,period.number,clock.value,start.down,start.distance,start.yardLine,start.yardsToEndzone,end.yardsToEndzone,homeScore,awayScore,statYardage,scoringPlay,scoreValue,type.text,type.abbreviation
0,GAME,0,900.0,0,0,0,0,65,0,0,0,False,0,Coin Toss,NaN
1,H.Butker kicks 65 yards from KC 35 to end zone...,1,900.0,0,0,35,65,70,0,0,0,False,0,Kickoff,K
2,(Shotgun) D.Henry left end to BLT 32 for 2 yar...,1,900.0,1,10,70,70,68,0,0,2,False,0,Rush,RUSH
3,(Shotgun) L.Jackson pass short right to Z.Flow...,1,859.0,2,8,68,68,73,0,0,-5,False,0,Pass Reception,REC
4,(Shotgun) L.Jackson pass short right to J.Hill...,1,835.0,2,13,73,73,71,0,0,2,False,0,Pass Reception,REC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,"(No Huddle, Shotgun) L.Jackson pass incomplete...",4,18.0,1,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
185,Timeout #4 by BLT at 00:10.,4,10.0,2,10,10,10,10,27,20,0,False,0,Timeout,TO
186,(Shotgun) L.Jackson pass incomplete short midd...,4,10.0,2,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
187,(Shotgun) L.Jackson pass short middle to I.Lik...,4,5.0,3,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN


In [54]:
len(plays)

189

## Testing the NNTRN 

https://gist.github.com/nntrn

In [9]:
import pandas as pd
import requests

In [23]:
import requests
import pandas as pd

# Fetch Athletes
df_athletes = pd.DataFrame(
    requests.get("https://sports.core.api.espn.com/v3/sports/football/nfl/athletes",
                 params={"page": 1, "limit": 20000}).json().get('items', [])
)

# Fetch Events
events = requests.get("https://partners.api.espn.com/v2/sports/football/nfl/events",
                      params={"limit": 1000, "dates": "20230101-20240101"}).json().get('events', [])

df_events = pd.DataFrame([{
    'event_id': e['id'], 'date': e['date'], 'name': e['name'],
    'home_team': (h := next(c for c in e['competitions'][0]['competitors'] if c['homeAway'] == 'home'))['team']['displayName'],
    'home_abbr': h['team']['abbreviation'], 'home_score': h['score']['value'],
    'away_team': (a := next(c for c in e['competitions'][0]['competitors'] if c['homeAway'] == 'away'))['team']['displayName'],
    'away_abbr': a['team']['abbreviation'], 'away_score': a['score']['value'],
    'winner': h['team']['abbreviation'] if h.get('winner') else a['team']['abbreviation'],
    'attendance': e['competitions'][0].get('attendance'),
    'status': e['competitions'][0]['status']['type']['detail']
} for e in events])

# Fetch Fantasy Players
df_fantasy = pd.DataFrame(
    requests.get("https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/2024/players",
                 params={"view": "kona_player_info"},
                 headers={'X-Fantasy-Filter': '{"games":{"limit":2000}}'}).json()
)

print(f"Athletes: {len(df_athletes)}, Events: {len(df_events)}, Fantasy: {len(df_fantasy)}")

Athletes: 19535, Events: 349, Fantasy: 2887


In [25]:
pd.set_option('display.max_columns', None)
df_athletes

,id,uid,guid,lastName,fullName,displayName,shortName,jersey,active,firstName,weight,displayWeight,height,displayHeight,age,dateOfBirth,experience,birthPlace,dateOfDeath,hand,middleName,citizenship,nickname
0,4246273,s:20~l:28~a:4246273,64cf2ccf-8d24-d1be-3ff4-d46499bdd9a6,[35],[35],[35],[35],35,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4246281,s:20~l:28~a:4246281,c6323fce-962f-a112-b136-9a4ed1d4282b,[Downed],[Downed],[Downed],[Downed],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4246289,s:20~l:28~a:4246289,21d50db4-55e3-a493-d76d-30b5a6786bd5,[Downed],[Downed],[Downed],[Downed],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4246247,s:20~l:28~a:4246247,5a011f43-549b-7b35-ad26-bb154b973384,[Touchback],[Touchback],[Touchback],[Touchback],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4246272,s:20~l:28~a:4246272,9efe3029-4e1f-8a43-2035-79e7479f251b,[Touchback],[Touchback],[Touchback],[Touchback],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19530,16424,s:20~l:28~a:16424,9128af51-f36e-32ba-1fdd-73fcbe2116bc,Zupancic,Mike Zupancic,Mike Zupancic,M. Zupancic,53,False,Mike,240.0,240 lbs,77.0,"6' 5""",35.0,1989-12-14T08:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19531,15462,s:20~l:28~a:15462,0d261997-24bf-5dd1-5e66-93272eed1a44,Zusevics,Markus Zusevics,Markus Zusevics,M. Zusevics,76,False,Markus,300.0,300 lbs,77.0,"6' 5""",36.0,1989-04-25T07:00Z,{'years': 3},NaN,NaN,NaN,NaN,NaN,NaN
19532,11317,s:20~l:28~a:11317,5a6c4114-bbc8-7941-f7f2-ebae32be0e09,Zuttah,Jeremy Zuttah,Jeremy Zuttah,J. Zuttah,64,False,Jeremy,300.0,300 lbs,76.0,"6' 4""",39.0,1986-06-01T07:00Z,{'years': 10},"{'city': 'Edison', 'state': 'NJ'}",NaN,NaN,NaN,NaN,NaN
19533,4294520,s:20~l:28~a:4294520,5114b190-78e6-daca-531b-be29d9cb2e72,Zylstra,Brandon Zylstra,Brandon Zylstra,B. Zylstra,13,False,Brandon,215.0,215 lbs,74.0,"6' 2""",32.0,1993-03-25T08:00Z,{'years': 5},"{'city': 'Spicer', 'state': 'MN', 'country': '...",NaN,NaN,NaN,NaN,NaN
